In [1]:
from gtfslib import GTFS
gtfs = GTFS("data/dart_gtfs_silverlineopening.zip")

import pandas as pd
from pathlib import Path

export_folder = Path("export/silverline_schedules")
export_folder.mkdir(parents=True, exist_ok=True)

stations = {
    "DFW Terminal B": "33473",
    "DFW Airport North": "33474",
    "Cypress Waters": "33595",
    "Downtown Carrollton": "34292",
    "Addison": "33596",
    "Knoll Trail": "33597",
    "UTD": "33598",
    "CityLine Bush": "26895",
    "12th Street": "34293",
    "Shiloh Road": "33600",
}
dates = [("openingday", "20251025"), ("weekday", "20251028"), ("weekend", "20251108")]

SILVER_LINE_ROUTE_ID = "26810"

In [16]:
gtfs.stops.to_csv(export_folder.parent / "stops.csv")

Generate departure timetable CSVs for each station

In [ ]:
for folder, date in dates:
    (export_folder / folder).mkdir(exist_ok=True)
    timetables = pd.DataFrame()
    for name, stop_id in stations.items():
        timetable = gtfs.build_stop_timetable(stop_id, [date])
        timetable["station"] = name
        timetable = timetable[["station", "trip_headsign", "departure_time", "block_id"]]
        timetable.columns = ["Station", "Destination", "Departure Time", "Train ID"]
        # CityLine Bush also serves light rail, so export all trains for that station separately
        if name == "CityLine Bush":
            timetable_all = timetable.copy()
            timetable_all.to_csv(export_folder / folder / f"{name.lower().replace(' ', '_')}_{folder}_{date}_alltrains.csv", index=False)
        timetable = timetable[timetable["Destination"].str.contains("SILVER LINE")]
        timetable.to_csv(export_folder / folder / f"{name.lower().replace(' ', '_')}_{folder}_{date}.csv", index=False)
        timetables = pd.concat([timetables, timetable])
    timetables.to_csv(export_folder / folder / f"all_stations_combined_{folder}_{date}.csv", index=False)

Generate combined destination timetables

In [7]:
from collections import defaultdict
import csv

def parse_time(t):
    h, m, _ = map(int, t.split(":"))
    if h >= 24:
        h -= 24
    suffix = "AM" if h < 12 else "PM"
    h %= 12
    if h == 0:
        h = 12
    return f"{h}:{m:02d} {suffix}"

for folder, date in dates:
    departures = defaultdict(lambda: defaultdict(list))
    for row in csv.DictReader((export_folder / folder / f"all_stations_combined_{folder}_{date}.csv").open()):
        departures[row["Destination"]][row["Station"]].append(row["Departure Time"])

    for destination, stations in departures.items():
        if "WEST" in destination:
            destination = "westbound"
        else:
            destination = "eastbound"
        rows = []
        for station, times in stations.items():
            row = [station] + list(map(parse_time, times))
            rows.append(row)
        if destination == "westbound":
            rows = rows[::-1]
        pd.DataFrame(rows).transpose().to_csv(export_folder / f"destination_{destination}_{folder}_{date}.csv", index=False, header=False)

In [2]:
# https://gtfsdata.ridetm.org/gtfs/fwtatransitdata.zip
tm_gtfs = GTFS("data/fwta_gtfs.zip")

In [37]:
tm_gtfs.stops.to_csv(export_folder.parent / "tm_stops.csv")
tm_gtfs.routes.to_csv(export_folder.parent / "tm_routes.csv")

In [14]:
tm_gtfs.build_stop_timetable("4156", ["20251025"])

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,date
114987,7689,140.0.2,924811,From Mercantile Station,<NA>,0,86471,33494,0,0,4:30:00,4:30:00,4156,5,<NA>,0,0,26.5394,1,20251025
114996,7689,140.0.2,924803,To DFW Airport Terminal B,<NA>,0,86470,33493,0,0,5:00:00,5:00:00,4156,8,<NA>,0,0,38.6591,1,20251025
115308,7689,140.0.2,924850,To Fort Worth T&P Station,<NA>,1,86470,33497,0,0,5:16:00,5:16:00,4156,2,<NA>,0,0,3.4625,1,20251025
115005,7689,140.0.2,924810,To DFW Airport Terminal B,<NA>,0,86473,33493,0,0,5:30:00,5:30:00,4156,8,<NA>,0,0,38.6591,1,20251025
115317,7689,140.0.2,924843,To Fort Worth T&P Station,<NA>,1,86473,33497,0,0,5:46:00,5:46:00,4156,2,<NA>,0,0,3.4625,1,20251025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115284,7689,140.0.2,924792,To DFW Airport Terminal B,<NA>,0,86472,33493,0,0,22:30:00,22:30:00,4156,8,<NA>,0,0,38.6591,1,20251025
115596,7689,140.0.2,924826,To Fort Worth T&P Station,<NA>,1,86472,33497,0,0,22:46:00,22:46:00,4156,2,<NA>,0,0,3.4625,1,20251025
115293,7689,140.0.2,924791,To DFW Airport Terminal B,<NA>,0,86473,33493,0,0,23:30:00,23:30:00,4156,8,<NA>,0,0,38.6591,1,20251025
115605,7689,140.0.2,924825,To Fort Worth T&P Station,<NA>,1,86473,33497,0,0,23:46:00,23:46:00,4156,2,<NA>,0,0,3.4625,1,20251025


In [22]:
pd.concat((gtfs.build_stop_timetable(stations["DFW Airport North"], ["20251025"]), tm_gtfs.build_stop_timetable("4156", ["20251025"]))).sort_values("departure_time")

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,date,trip_short_name,wheelchair_accessible,bikes_allowed
114987,7689,140.0.2,924811,From Mercantile Station,0,86471,33494,4:30:00,4:30:00,4156,5,<NA>,0,0,26.5394,1,20251025,<NA>,0,0
114996,7689,140.0.2,924803,To DFW Airport Terminal B,0,86470,33493,5:00:00,5:00:00,4156,8,<NA>,0,0,38.6591,1,20251025,<NA>,0,0
115308,7689,140.0.2,924850,To Fort Worth T&P Station,1,86470,33497,5:16:00,5:16:00,4156,2,<NA>,0,0,3.4625,1,20251025,<NA>,0,0
115005,7689,140.0.2,924810,To DFW Airport Terminal B,0,86473,33493,5:30:00,5:30:00,4156,8,<NA>,0,0,38.6591,1,20251025,<NA>,0,0
115317,7689,140.0.2,924843,To Fort Worth T&P Station,1,86473,33497,5:46:00,5:46:00,4156,2,<NA>,0,0,3.4625,1,20251025,<NA>,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115605,7689,140.0.2,924825,To Fort Worth T&P Station,1,86473,33497,23:46:00,23:46:00,4156,2,<NA>,0,0,3.4625,1,20251025,<NA>,0,0
986359,26810,13,8804046,SILVER LINE - EAST - SHILOH ROAD,0,66401,146593,24:04:00,24:04:00,33474,2,<NA>,0,0,3.5183,1,20251025,<NA>,<NA>,<NA>
986086,26810,13,8803753,SILVER LINE - WEST - DFW AIRPORT,1,66403,146594,24:11:00,24:11:00,33474,9,<NA>,0,0,41.1508,1,20251025,<NA>,<NA>,<NA>
986369,26810,13,8804047,SILVER LINE - EAST - SHILOH ROAD,0,66403,146593,24:34:00,24:34:00,33474,2,<NA>,0,0,3.5183,1,20251025,<NA>,<NA>,<NA>


In [60]:
gtfs.build_stop_timetable("34304", ["20260125"])

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,date
913300,26730,4,8711011,236 PARKER ROAD STATION,1,23601,146161,05:03:19,05:03:19,34304,5,<NA>,0,0,1.3791,0,20260125
913828,26730,4,8711019,236 PARKER ROAD STATION,1,23401,146161,06:03:19,06:03:19,34304,5,<NA>,0,0,1.3791,0,20260125
913366,26730,4,8711012,236 PARKER ROAD STATION,1,23602,146161,07:03:36,07:03:36,34304,5,<NA>,0,0,1.3791,0,20260125
913894,26730,4,8711020,236 PARKER ROAD STATION,1,23402,146161,08:03:36,08:03:36,34304,5,<NA>,0,0,1.3791,0,20260125
913432,26730,4,8711013,236 PARKER ROAD STATION,1,23601,146161,09:03:36,09:03:36,34304,5,<NA>,0,0,1.3791,0,20260125
913960,26730,4,8711021,236 PARKER ROAD STATION,1,23603,146161,09:43:36,09:43:36,34304,5,<NA>,0,0,1.3791,0,20260125
914026,26730,4,8711022,236 PARKER ROAD STATION,1,23401,146161,10:23:36,10:23:36,34304,5,<NA>,0,0,1.3791,0,20260125
914092,26730,4,8711023,236 PARKER ROAD STATION,1,23622,146161,11:03:36,11:03:36,34304,5,<NA>,0,0,1.3791,0,20260125
913498,26730,4,8711014,236 PARKER ROAD STATION,1,23403,146161,11:43:36,11:43:36,34304,5,<NA>,0,0,1.3791,0,20260125
914158,26730,4,8711024,236 PARKER ROAD STATION,1,23422,146161,12:23:36,12:23:36,34304,5,<NA>,0,0,1.3791,0,20260125


In [20]:
transfer_stations = {
    "DFW Terminal B": ["TM:4158"],
    "DFW Airport North": ["TM:4156"],
    "Cypress Waters": [],
    "Downtown Carrollton": ["29817", "33299"],
    "Addison": ["33245"],
    "Knoll Trail": ["34304", "34305"],  # 236, both directions
    "UTD": ["34288"],
    "CityLine Bush": ["26895", "33260"],  # first serves light rail
    "12th Street": ["33599", "34289"],
    "Shiloh Road": [],
}

for station, stop_ids in transfer_stations.items():
    print(f"{station} Station Transfers:")
    stops_to_look_at = [stations[station]] + stop_ids

    combined_tt = gtfs.build_stop_timetable(stations[station], ["20251028"])
    combined_tt["route_name"] = combined_tt["route_id"].map(lambda rid: gtfs._routes_by_id.at[rid, "route_short_name"])

    unique_rts = set()
    for stop_id in stop_ids:
        if stop_id.startswith("TM:"):
            stop_id = stop_id[3:]
            g = tm_gtfs
        else:
            g = gtfs
        tt = g.build_stop_timetable(stop_id, ["20251028"])
        tt["route_name"] = tt["route_id"].map(lambda rid: g._routes_by_id.at[rid, "route_short_name"])
        if stop_id != stations[station]:
            combined_tt = pd.concat((combined_tt, tt))
        
        # Aggregate by route type
        routes = tt["route_id"].unique()
        routes = routes[routes != SILVER_LINE_ROUTE_ID]
        unique_rts.update(routes)
    
    if not len(unique_rts):
        print("  (none)")
    else:
        for route_type, rts in g._routes_by_id.loc[list(unique_rts)].groupby("route_type"):
            short_names = sorted(rts["route_short_name"].tolist())
            if route_type == 0 or route_type == 2:
                rt_name = "Rail"
            elif route_type == 3:
                rt_name = "Bus"
            else:
                rt_name = f"Other"
            print(f"  {rt_name}: ", end="")
            print(", ".join(short_names))

    combined_tt["departure_time"] = combined_tt["departure_time"].map(lambda t: f"{int(t.split(':')[0]):02d}:{t.split(':')[1]}:{t.split(':')[2]}")
    combined_tt = combined_tt.sort_values("departure_time")
    combined_tt = combined_tt[combined_tt["drop_off_type"] != 1]  # Exclude "no drop off" trips
    combined_tt = combined_tt[["route_name", "trip_headsign", "direction_id", "departure_time"]]
    print(combined_tt[["route_name", "trip_headsign", "direction_id"]].drop_duplicates().sort_values("route_name").to_string(index=False, header=["Route", "Destination", "Dir"]))
    print(combined_tt.to_string(index=False))

DFW Terminal B Station Transfers:
  Rail: TEXRail
  Route                      Destination Dir
 SILVER SILVER LINE - WEST - DFW AIRPORT   1
 SILVER SILVER LINE - EAST - SHILOH ROAD   0
TEXRail          From Mercantile Station   0
TEXRail        To DFW Airport Terminal B   0
TEXRail        To Fort Worth T&P Station   1
TEXRail            To Mercantile Station   1
route_name                    trip_headsign  direction_id departure_time
    SILVER SILVER LINE - WEST - DFW AIRPORT             1       04:18:00
    SILVER SILVER LINE - EAST - SHILOH ROAD             0       04:28:00
   TEXRail          From Mercantile Station             0       04:36:00
   TEXRail        To DFW Airport Terminal B             0       05:06:00
   TEXRail        To Fort Worth T&P Station             1       05:10:00
    SILVER SILVER LINE - WEST - DFW AIRPORT             1       05:18:00
    SILVER SILVER LINE - EAST - SHILOH ROAD             0       05:28:00
   TEXRail        To DFW Airport Terminal B        